In [1]:
import os

import bs4
import dotenv
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

dotenv.load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [9]:
# Load data in pandas, only first 10 rows
import pandas as pd

df = pd.read_csv("./data/dutch-news-articles.csv", nrows=10, encoding="utf-8")
df.head()

,datetime,title,content,category,url
0,2010-01-01 00:49:00,Enige Litouwse kerncentrale dicht,De enige kerncentrale van Litouwen is oudjaars...,Buitenland,https://nos.nl/artikel/126231-enige-litouwse-k...
1,2010-01-01 02:08:00,Spanje eerste EU-voorzitter onder nieuw verdrag,Spanje is met ingang van vandaag voorzitter va...,Buitenland,https://nos.nl/artikel/126230-spanje-eerste-eu...
2,2010-01-01 02:09:00,Fout justitie in Blackwater-zaak,Vijf werknemers van het omstreden Amerikaanse ...,Buitenland,https://nos.nl/artikel/126233-fout-justitie-in...
3,2010-01-01 05:14:00,"Museumplein vol, minder druk in Rotterdam",Het Oud en Nieuwfeest op het Museumplein in Am...,Binnenland,https://nos.nl/artikel/126232-museumplein-vol-...
4,2010-01-01 05:30:00,Obama krijgt rapporten over aanslag,President Obama heeft de eerste rapporten gekr...,Buitenland,https://nos.nl/artikel/126236-obama-krijgt-rap...


# RAG against my machine

In [29]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./data/dutch-news-articles.csv", encoding="utf-8", source_column="url", metadata_columns=["title", "datetime", "category"])

data = loader.load()
print(data[0].page_content[:500])  # Example of the first 500 characters of the first document

content: De enige kerncentrale van Litouwen is oudjaarsavond om 23.00 uur buiten gebruik gesteld. Dat verliep zonder problemen, aldus de directeur. Litouwen beloofde al in 2004 om de centrale te sluiten in ruil voor toetreding tot de Europese Unie. De EU wilde sluiting omdat de kerncentrale bij de stad Visiginas mogelijk niet veilig was. Nucleaire ramp De centrale is een grotere versie van die bij Tsjernobyl. Die ontplofte in 1986 en veroorzaakte een nucleaire wolk die over een groot deel van Eu


In [30]:
data[42]

Document(page_content="content: Een vulkaanuitbarsting in het oosten van Congo bedreigt de leefomgeving van een aantal zeldzame chimpansees. Volgens de beheerders van het Virunga National Park wordt de situatie nauwlettend in de gaten gehouden. Dat gebeurt onder meer met helikopters van de VN. De vulkaan Nyamulagira barstte vanochtend vroeg uit. De lava stroomt over de oostelijke flank van de berg, in de richting van het gebied waar de veertig chimpansees leven. In het wildpark in Congo zijn ook nog ongeveer 200 zeldzame berggorilla's. Hun leefgebied ten zuiden van de vulkaan wordt niet bedreigd. Actiefste In het gebied dat op de werelderfgoedlijst staat van de Unesco wonen vrijwel geen mensen. De vulkaan in de buurt van de oostelijke stad Goma is één van de actiefste van Afrika. Sinds 1882 is hij meer dan 35 keer tot uitbarsting gekomen. De laatste keer was vier jaar geleden.\nurl: https://nos.nl/artikel/126498-vulkaan-congo-bedreigt-chimpansees.html", metadata={'source': 'https://n

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(data[:10])

print(f"{len(splits)} splits in total")
print(f"Metadata: {splits[4].metadata}")
print(f"Contents:\n\n{splits[4].page_content}")

21 splits in total
Metadata: {'source': 'https://nos.nl/artikel/126233-fout-justitie-in-blackwater-zaak.html', 'row': 2, 'title': 'Fout justitie in Blackwater-zaak', 'datetime': '2010-01-01 02:09:00', 'category': 'Buitenland', 'start_index': 0}
Contents:

content: Vijf werknemers van het omstreden Amerikaanse beveilingingsbedrijf Blackwater gaan vrijuit voor hun aandeel in een dodelijke schietpartij in Bagdad. De mannen werden verdacht van het doodschieten van zeventien Irakese burgers in 2007. Een rechter in Washington bepaalde dat het bewijs tegen hen niet rechtmatig is verkregen en heeft de zaak daarom niet-ontvankelijk verklaard. Beloofd De verdachten was door rechercheurs beloofd dat hun verklaringen alleen gebruikt zouden worden voor het onderzoek naar de schietpartij en dat die geen rol zouden spelen in een strafzaak. Justitie had dus aanvullend bewijs moeten vergaren, maar is daar volgens de rechter niet in geslaagd. De schietpartij op een plein in Bagdad veroorzaakte wereldwij

In [32]:
# sentence-transformers/all-MiniLM-L6-v2
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

In [33]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Welk bedrijf kwam recent in het nieuws?")

print(f"{len(retrieved_docs)} retrieved docs")
print(f"Contents:\n\n{retrieved_docs[0].page_content[:100]}")

6 retrieved docs
Contents:

Ook in Kamp Holland hebben de Nederlandse militairen de jaarwisseling gevierd. Aan de zijkant van de
